# Modular Exponentiation
## General Notes
    - Looking to solve X^Y mod N
    - Central subroutine of **Shor's period finding algorithm** used for integer factorization

## Goal
Implement a quantum circuit for modular exponentiation from scratch.

# Limitations
    - Unless stated otherwise, you are only allowed to use the gates X, CNOT, CCNOT and Multicontrolled-NOT. See Appendix A for an example of use of the Multicontrolled-NOT gate.

    - Can implement additional auxiliary functions as long as the functions below are implemented.

    - For each implemented function, please give evidence that the implementation is correct by:
        - initializing each input register with some number up to 4 bits, 
        - each auxiliary register with |0>
        - measuring the output register to verify if the value is as expected

    - Reuse qubits from auxiliary registers as much as possible.
        - It is crucial that auxiliary registers are equal to |0> both at the beginning and at the end of computation of each function

In [1]:
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import SamplerV2 as Sampler
from qiskit import QuantumCircuit
from qiskit.providers.basic_provider import BasicSimulator
from qiskit import transpile
import numpy as np

# 1.1 Initialization
The function `set_bits(circuit,A,X)` initializes the bits of register `A` with the binary string `X`.

For each i in `len(X)`, if `X[i]=1`, then the function applies the **X**-gate to `A[i]`.

Otherwise, it does nothing.

Assume `len(A)=len(X)`.

If `qubits = [2,4,3,7,5]` and `X = 01011`, the **X**-gate is applied to qubits 4, 7, and 5

In [ ]:
def set_bits(circuit,A,X):

    # Width is determined by A
    w = len(A)
    
    # Check if X is read-in as a string or an integer
    # If X is written as int, X = 11
    if isinstance(X, int):

        # Check not negative
        if X < 0:
            raise ValueError("X must be a positive integer.")
        
        # Convert to binary
        X = bin(X)
        # Remove leading 0b
        X = X[2:]
        # Pad
        X = X.zfill(w)

    elif isinstance(X, str):
            
            # String handling
            if X.startswith("0b"):

                # X has already been wrapped by bin(), ie X = bin(11)
                # Strip 0b
                X = X[2:]
                # Pad
                X = X.zfill(w)
        
            else:

                # X is just the string "01011" or "1011"
                # and we are assuming that input for the decimal number "1,011" will jsut be written as X=1011 not to be confused w binary
                X = X.zfill(w)

    else:

        raise TypeError("X must be an int or str")
    
    if len(X) > w:
        raise ValueError("Binary value doesn't fit in target register")

    # Enforce width
    X = X.zfill(w)

    # Apply X gate
    for i in range(w):
        if X[i]=="1":
            circuit.x(A[i])
        

    circuit.barrier()

    return circuit

In [ ]:
# Check 1.1 initialization
circuit = QuantumCircuit(8, 0) 
A = [2,4,3,7,5] # qubits
# X = 01011, which is 11 in decimal
# X = bin(11) 
X = "1011"

# print(X)
# print(len(X))
# print(A)
# print(len(A))

set_bits(circuit,A,X)
print(circuit)

           ░ 
q_0: ──────░─
           ░ 
q_1: ──────░─
           ░ 
q_2: ──────░─
           ░ 
q_3: ──────░─
     ┌───┐ ░ 
q_4: ┤ X ├─░─
     ├───┤ ░ 
q_5: ┤ X ├─░─
     └───┘ ░ 
q_6: ──────░─
     ┌───┐ ░ 
q_7: ┤ X ├─░─
     └───┘ ░ 


# 1.2 Copy
The function `copy(circuit,A,B)` copies the binary string `bin(A)` to register B.

Assume that `len(A)=len(B)` and before application of function, B is initialized to |0>

**Hint: use CNOT gates**

In [56]:
def copy(circuit,A,B):

    # Copy binary sting bin(A) to register B using CNOT gates
    for i in range(len(A)):
        circuit.cx(A[i], B[i])

    return circuit

In [57]:
# Check 1.2 copy()
circuit = QuantumCircuit(8, 4)
A = [0, 1, 2, 3]
B = [4, 5, 6, 7]

# Use set_bits() (testing different inputs just in case)
set_bits(circuit, A, 11)
# set_bits(circuit, A, "1011")
# set_bits(circuit, A, "01011")

# Copy A to B
copy(circuit,A,B)

print(A)
print(B)
print(circuit)

[0, 1, 2, 3]
[4, 5, 6, 7]
     ┌───┐ ░                     
q_0: ┤ X ├─░───■─────────────────
     └───┘ ░   │                 
q_1: ──────░───┼────■────────────
     ┌───┐ ░   │    │            
q_2: ┤ X ├─░───┼────┼────■───────
     ├───┤ ░   │    │    │       
q_3: ┤ X ├─░───┼────┼────┼────■──
     └───┘ ░ ┌─┴─┐  │    │    │  
q_4: ──────░─┤ X ├──┼────┼────┼──
           ░ └───┘┌─┴─┐  │    │  
q_5: ──────░──────┤ X ├──┼────┼──
           ░      └───┘┌─┴─┐  │  
q_6: ──────░───────────┤ X ├──┼──
           ░           └───┘┌─┴─┐
q_7: ──────░────────────────┤ X ├
           ░                └───┘
c: 4/════════════════════════════
                                 


# 1.3 Full Adder
The function `full_adder(circuit,a,b,r,c_in,c_out,AUX)` implements a full adder.

Registers:

`a` and `b` store the bits to the added

`c_in` stores the carry-in bit

`c_out` stores the carry-out bit

`r` stores the result of the sum

`AUX` is the auxiliary register

In [58]:
def full_adder(circuit,a,b,r,c_in,c_out,AUX):

    # Sum with XOR: r ^ (a ^ b ^ c_in)
    # r <- r ^ a
    circuit.cx(a, r)
    # r <- r ^ b
    circuit.cx(b, r)
    # r <- r ^ c_in
    circuit.cx(c_in, r)

    # Carry with CCNOT: c_out <- c_out ^ (a & b) ^ (a & c_in) ^ (b & c_in)
    circuit.ccx(a, b, c_out)
    circuit.ccx(a, c_in, c_out)
    circuit.ccx(b, c_in, c_out)

    return circuit

In [61]:
# 1.3 Full Adder check
circuit = QuantumCircuit(5, 2)

# Assign qubits
a = 0
b = 1
c_in = 2
r = 3
c_out = 4
AUX = []

# Initialize a = 1, b = 1, c_in = 0
circuit.x(a)
circuit.x(b)

# Run full adder
full_adder(circuit, a, b, r, c_in, c_out, AUX)

# Results
circuit.measure(r, 0)
circuit.measure(c_out, 1)

simulator = BasicSimulator()
result = simulator.run(circuit).result()
counts = result.get_counts()

print(counts)
print(circuit)

{'10': 1024}
     ┌───┐                                    
q_0: ┤ X ├──■──────────────■────■─────────────
     ├───┤  │              │    │             
q_1: ┤ X ├──┼────■─────────■────┼───────■─────
     └───┘  │    │         │    │       │     
q_2: ───────┼────┼────■────┼────■───────■─────
          ┌─┴─┐┌─┴─┐┌─┴─┐  │    │  ┌─┐  │     
q_3: ─────┤ X ├┤ X ├┤ X ├──┼────┼──┤M├──┼─────
          └───┘└───┘└───┘┌─┴─┐┌─┴─┐└╥┘┌─┴─┐┌─┐
q_4: ────────────────────┤ X ├┤ X ├─╫─┤ X ├┤M├
                         └───┘└───┘ ║ └───┘└╥┘
c: 2/═══════════════════════════════╩═══════╩═
                                    0       1 


# 1.4 Addition
The function `add(circuit,A,B,R,AUX)` implements a circuit that adds `number(A)` to `number(B)` and stores the result at register `R`.

Assume `len(A)==len(B)==lent(R)`

The circuit is obtained by creating as cascade of `full_adder` circuits.

The carry bits are part of the auxiliary register AUX.

Note the carry-in bit of the first adder (from right to left) is set to 0.

In [62]:
def add(circuit,A,B,R,AUX):

    # Make sure lengths are the same

    # R <- R ^ (number(A) + number(B))
    # AUX is where carry bits live
    for i in range(len(A)):

        c_in = AUX[i]
        c_out = AUX[i+1]

        full_adder(circuit, A[i], B[i], R[i], c_in, c_out, AUX)

    return circuit

In [64]:
# 1.4 Check Addition
circuit = QuantumCircuit(17, 4)

# Number of 4-bit numbers
n = 4

# Registers
A = [0, 1, 2, 3]
B = [4, 5, 6, 7]
R = [8, 9, 10, 11]
AUX = [12, 13, 14, 15, 16]

# Let A = 3 (0011)
circuit.x(A[0])
circuit.x(A[1])

# Let B = 5 (0101)
circuit.x(B[0])
circuit.x(B[2])

# Compute addition
add(circuit, A, B, R, AUX)

for i in range(n):
    circuit.measure(R[i], i)

simulator = BasicSimulator()
result = simulator.run(circuit).result()
counts = result.get_counts()

print(counts)
print(circuit)

{'1000': 1024}
      ┌───┐                                                                 »
 q_0: ┤ X ├────────────■────────────────────────────────────────────■───────»
      ├───┤            │                                            │       »
 q_1: ┤ X ├────────────┼────■───────────────────────────────────────┼────■──»
      └───┘            │    │                                       │    │  »
 q_2: ───────■─────────┼────┼────────────────────────■──────────────┼────┼──»
             │         │    │                        │              │    │  »
 q_3: ───────┼────■────┼────┼────────────────────────┼────■─────────┼────┼──»
      ┌───┐  │    │    │    │                        │    │         │    │  »
 q_4: ┤ X ├──┼────┼────┼────┼──────────────■─────────┼────┼─────────■────┼──»
      └───┘  │    │    │    │              │         │    │         │    │  »
 q_5: ───────┼────┼────┼────┼──────────────┼────■────┼────┼─────────┼────■──»
      ┌───┐  │    │    │    │              │    │

# 1.5 Subtraction
The function `subtract(circuit,A,B,R,AUX)` implements a circuit that subtracts `Number(B)` from `Number(A)` and stores the result in the register `R`.

Assume that `len(A)=len(B)=len(R)`.

Such a circuit can be obtained by negating each bit stores in B, and applying the adder circuit with the first carry-in bit set to 1 instead of 0.

In [ ]:
def subtract(circuit,A,B,R,AUX):
    pass